<변경점>  
Arcface 사람별로 첫 20frame만 적용  
segmentation으로 사람 경계선 땀

1. 사람 + 총 + 칼 인식    
2. Arcface를 모든 사람에 대해 적용(Me, NotME). 첫 20프레임만  
3. NotMe이면서 사람과 총 혹은 칼의 bounding box가 겹치는 경우에는 위험인으로 분류 -> 한 번 위험인으로 분류된 사람의 ID는 dangerous_ids로 관리됨 -> dangerous_ids 중 하나에 해당하는 사람은 위에 "Dangerous person"이라고 뜸
4. pose estimation을 dangerous person에 대해 적용(왼팔들기, 오른팔들기, 양팔들기)  
5. 출력화면 사이즈를 키움  
6. 버그수정: 초기에 무기와 교차하여 dangerous_ids에 들어갔더라도, ArcFace로 내 얼굴임이 확인되면(=나 자신이 무기를 소지한 상황) 그 사람을 위험 인물에서 제외  
7. Kobukki robot이 dangerous_ids 내에 있는 사람을 쫓아가도록 하면 될 듯
8. warning 해결: pip install --upgrade albumentations
9. 사람인 경우에 segmentation으로 윤곽선 치기(성공)


In [8]:
#1. 얼굴 학습시키는 부분
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis

def initialize_arcface():
    app = FaceAnalysis(name="buffalo_s")  # ArcFace 모델 (buffalo_s은 기본 권장)
    app.prepare(ctx_id=-1, det_size=(640, 640))  # GPU: ctx_id=0, CPU: -1
    return app

def get_face_embedding(app, image_bgr):
    # ArcFace의 app.get()은 BGR 형식으로 이미지를 받기도 합니다.
    # 만약 RGB가 필요하면 cvtColor로 변환하세요.
    faces = app.get(image_bgr)
    if len(faces) > 0:
        return faces[0].embedding  # 첫 번째 얼굴의 임베딩
    else:
        return None

def generate_average_embedding(app, folder_path):
    embeddings = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"이미지 로드 실패: {img_path}")
                continue
            
            embedding = get_face_embedding(app, image)
            if embedding is not None:
                embeddings.append(embedding)
            else:
                print(f"얼굴 검출 실패: {img_path}")
    
    if len(embeddings) == 0:
        raise ValueError("임베딩을 하나도 생성하지 못했습니다.")
    
    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding

if __name__ == "__main__":
    app = initialize_arcface()
    # 내 얼굴 사진 폴더
    my_face_folder = "C:/Users/idea0/EE101/Jongsul/myface"  
    my_face_embedding = generate_average_embedding(app, my_face_folder)
    np.save("my_face_embedding.npy", my_face_embedding)  # 필요 시 저장
    print("내 얼굴 평균 임베딩 생성 완료.")

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127

In [9]:
import cv2
import mediapipe as mp
import time
import torch
import numpy as np
import warnings

from ultralytics import YOLO
from sklearn.metrics.pairwise import cosine_similarity
from insightface.app import FaceAnalysis
# Deep SORT
from deep_sort_realtime.deepsort_tracker import DeepSort

# 버전 업그레이드 권장 무시
warnings.filterwarnings("ignore", category=FutureWarning)

############################
# 1) 모델 및 함수 초기화
############################
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# (A) YOLO **세그멘테이션** 모델 (사람=0, 총=1, 칼=2)
#model = YOLO("/home/dev/runs/detect/train55/weights/epoch180.pt").to(device)
model = YOLO("C:/Users/idea0/EE101/Jongsul/Yolomodels/epoch180.pt").to(device)

model_seg = YOLO("yolov8n-seg.pt").to(device)

#face_model = YOLO("/home/dev/runs/detect/train34/weights/best.pt").to(device)
face_model = YOLO("C:/Users/idea0/EE101/Jongsul/Yolomodels/best.pt").to(device)

# ArcFace
arc_app = FaceAnalysis(name="buffalo_s")
arc_app.prepare(ctx_id=-1, det_size=(640,640))
my_face_embedding = np.load("my_face_embedding.npy")

def get_face_embedding(arc_app, face_img_bgr):
    faces = arc_app.get(face_img_bgr)
    if len(faces) == 0:
        return None
    return faces[0].embedding

def is_my_face(face_embedding, my_embedding, threshold=0.4):
    sim = cosine_similarity([face_embedding], [my_embedding])[0][0]
    return (sim > threshold), sim

# Pose: Dangerous person만 크롭 영상에 대해 Pose
mp_pose = mp.solutions.pose
pose_danger = mp_pose.Pose(
    static_image_mode=True,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_drawing = mp.solutions.drawing_utils

# 팔 들기 판별용(부위 인덱스)
LEFT_SHOULDER = 11
RIGHT_SHOULDER = 12
LEFT_WRIST = 15
RIGHT_WRIST = 16

def is_arm_raised(shoulder_y, wrist_y, threshold=0.05):
    return wrist_y < (shoulder_y - threshold)

# 바운딩박스 overlap
def boxes_overlap(boxA, boxB):
    (x1A, y1A, x2A, y2A) = boxA
    (x1B, y1B, x2B, y2B) = boxB
    overlap_x = not (x2A < x1B or x2B < x1A)
    overlap_y = not (y2A < y1B or y2B < y1A)
    return overlap_x and overlap_y

############################
# 2) DeepSORT 초기화
############################
tracker = DeepSort(max_age=30,
                   n_init=3,
                   nms_max_overlap=1.0,
                   embedder='mobilenet',
                   half=True,
                   embedder_gpu=True)

############################
# 3) 메인 루프
############################
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

prev_time = time.time()

# 위험 인물(무기와 교차 & 내가 아닌) track_id
dangerous_ids = set()
sim = 0

# --- (A) ArcFace 저장 구조: 각 트랙별 ---
track_me_status = {}       # tid -> True/False/None
track_arcface_count = {}   # tid -> 몇 번 ArcFace 시도했나
MAX_ARCFACE_FRAMES = 20    # 최대 ArcFace 적용 프레임 수

# window resize
window_name = "DeepSORT + YOLO(SEG) + ArcFace + Pose(DangerOnly)"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 960, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    # (1) YOLO 세그멘테이션 추론
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(rgb_frame , conf=0.5)  # conf threshold = 0.5
    results_seg = model_seg(rgb_frame, conf=0.5)

    # YOLOv8 세그멘테이션 결과
    det = results[0]
    boxes2 = det.boxes
    masks2 = results_seg[0].masks  # 세그멘테이션 마스크 정보(각 객체별)

    person_detections = []
    weapon_boxes = []

    if boxes2 is not None:
        for i, box in enumerate(boxes2):
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls[0])
            conf = float(box.conf[0])

            # 바운딩박스 시각화
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
            label = f"{model.names[class_id]}: {conf:.2f}"
            cv2.putText(frame, label, (x1,y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

            # (A) 세그멘테이션 윤곽선 그리기 (사람=0, 총=1, 칼=2)
            if masks2 is not None and class_id in [0,1,2]:
                single_mask = masks2.data[i].cpu().numpy()  # shape=(H,W)
                mask_bin = (single_mask > 0.5).astype(np.uint8)
                contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # 클래스별 색상 예시
                if class_id == 0:   # person
                    color_seg = (0,255,255)
                elif class_id == 1: # gun
                    color_seg = (0,0,255)
                elif class_id == 2: # knife
                    color_seg = (255,0,0)
                else:
                    color_seg = (255,255,255)

                cv2.drawContours(frame, contours, -1, color_seg, 2)

            # (B) 바운딩박스 정보 → DeepSORT
            if class_id == 0:
                w = x2 - x1
                h = y2 - y1
                person_detections.append(((x1,y1,w,h), conf, 0))
            elif class_id in [1,2]:
                weapon_boxes.append((x1,y1,x2,y2))

    # (2) DeepSORT update
    tracks = tracker.update_tracks(person_detections, frame=rgb_frame)
    tracked_boxes = []
    for t in tracks:
        if not t.is_confirmed() or t.time_since_update>1:
            continue
        tid = t.track_id
        l,t_,r,b_ = map(int, t.to_ltrb())
        tracked_boxes.append((tid, l,t_,r,b_))
        cv2.putText(frame, f"ID:{tid}", (l-10,t_-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255),2)

    # 3) ArcFace: 각 트랙에 대해 최대 10프레임만 시도
    for (tid, px1, py1, px2, py2) in tracked_boxes:
        # (A) 초기화
        if tid not in track_me_status:
            track_me_status[tid] = False  # 아직 확정X
        if tid not in track_arcface_count:
            track_arcface_count[tid] = 0  # ArcFace 시도 횟수

        # 현재 me_status
        me_status = track_me_status[tid]

        # 이미 me_status가 True면 더 이상 ArcFace X
        # 아니면 ArcFace 시도 횟수 < MAX_ARCFACE_FRAMES인 경우에만 시도
        if me_status is False and track_arcface_count[tid] < MAX_ARCFACE_FRAMES:
            track_arcface_count[tid] += 1
            print("running Arcface...")

            # Face detection
            px1_clamped = max(0, px1)
            face_results = face_model(frame[py1:py2, px1_clamped:px2], verbose=False)

            person_crop = None
            if len(face_results) > 0 and len(face_results[0].boxes) > 0:
                for fbox in face_results[0].boxes:
                    if int(fbox.cls) == 2:  # 가정: face_model에서 cls=2가 'face'
                        fx1, fy1, fx2, fy2 = map(int, fbox.xyxy[0])
                        person_crop = frame[py1+fy1:py1+fy2, px1_clamped+fx1:px1_clamped+fx2]
                        break

            if person_crop is not None and person_crop.size>0:
                face_embedding = get_face_embedding(arc_app, person_crop)
                if face_embedding is not None:
                    same_person, sim = is_my_face(face_embedding, my_face_embedding, threshold=0.4)
                    if same_person:
                        track_me_status[tid] = True

     
        if me_status:
            text_arc= f"             Me(sim={sim:.2f})"
            color=(0,255,0)
            if tid in dangerous_ids:             # 이미 me가 되었으니 dangerous_ids에서 제거
                dangerous_ids.remove(tid)
        else:
            text_arc= f"             NotMe(sim={sim:.2f})"
            color=(0,0,255)
            
        cv2.putText(frame, text_arc, (px1, py1-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color,2)

        # else: 이미 True/False이거나 시도 횟수 끝 -> 아무것도 안함
        
    # (4) 무기 교차 & 내가 아닌 => dangerous_ids
    for (tid, px1, py1, px2, py2) in tracked_boxes:
        if not track_me_status[tid]:
            pbox = (px1, py1, px2, py2)
            for wb in weapon_boxes:
                if boxes_overlap(pbox, wb):
                    dangerous_ids.add(tid)
                    break

    # (5) Dangerous => Pose
    for (tid, px1, py1, px2, py2) in tracked_boxes:
        if tid in dangerous_ids:
            sub = frame[py1:py2, px1:px2]
            if sub.size==0:
                continue
            c_rgb = cv2.cvtColor(sub, cv2.COLOR_BGR2RGB)
            pose_result = pose_danger.process(c_rgb)
            if pose_result.pose_landmarks:
                lms = pose_result.pose_landmarks.landmark
                sub_w = px2 - px1
                sub_h = py2 - py1

                left_shoulder_y = lms[11].y
                right_shoulder_y = lms[12].y
                left_wrist_y = lms[15].y
                right_wrist_y = lms[16].y

                left_arm_up = (left_wrist_y< (left_shoulder_y-0.05))
                right_arm_up= (right_wrist_y<(right_shoulder_y-0.05))
                action_text=""
                if left_arm_up and right_arm_up:
                    action_text="both arms up"
                elif left_arm_up:
                    action_text="left arm up"
                elif right_arm_up:
                    action_text="right arm up"
                else:
                    action_text="do nothing"

                for lm in lms:
                    cx = px1+int(lm.x*sub_w)
                    cy = py1+int(lm.y*sub_h)
                    cv2.circle(frame,(cx,cy),3,(0,255,255),-1)
                cv2.putText(frame, f"Dangerous person: {action_text}",
                            (px1, py1+20), cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)

    # FPS
    now = time.time()
    fps = 1.0/(now - prev_time)
    prev_time=now
    cv2.putText(frame,f"FPS:{fps:.2f}",(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

    cv2.imshow(window_name,frame)
    if cv2.waitKey(1)&0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127


0: 480x640 1 person, 133.0ms
Speed: 3.0ms preprocess, 133.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personnnn, 98.0ms
Speed: 1.0ms preprocess, 98.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 127.0ms
Speed: 2.0ms preprocess, 127.0ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personnnn, 119.0ms
Speed: 3.0ms preprocess, 119.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 140.0ms
Speed: 5.0ms preprocess, 140.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personnnn, 92.0ms
Speed: 1.0ms preprocess, 92.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 127.0ms
Speed: 3.0ms preprocess, 127.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 personnnn, 99.0ms
Speed: 1.0ms preprocess, 99.0ms inference, 1.0ms postprocess per im

In [1]:
import torch

print("CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())

CUDA Version: None
CUDA Available: False
